# Neural Posterior Estimation

May the machine spirit bless me in this endeavour

In [ ]:
from JHPY import *
import torch.nn as nn


In [ ]:
outputs = generate_sine_data(data_points=1024, mean=10, std=2)

train_loader = outputs["Train_Loader"]
val_loader = outputs["Val_Loader"]
test_loader = outputs["Test_Loader"]